# Regularization and MAP

The supervised parametric machine learning framework we have been using can be summarized as follows:

1.  We have some data $(x_i,y_i)$ for $i \in \{1,2,3,\dots, n\}$. where $x_i \in \mathbb{R}^p$ and $y_i \in \mathbb{R}$.
2.  We presume that there is a true functional relationship $f: \mathbb{R}^p \to \mathbb{R}$, but that this is corrupted by some error term:

$$
Y = f(X) + \epsilon 
$$

3.  We cannot search over **all** functions for a candidate $\hat{f}$, so we instead specify some collection of functions $f_\beta$ which depend on some parameters $\beta$.
4.  We introduce a loss function $\ell(\beta)$ to measure how good a job $f_\beta$ does on the training data.  $\ell$ takes $\beta$ as input and returns a non-negative real number as output.
5.  We choose $\hat{\beta}$ so that $\ell(\hat{\beta})$ is minimized or, if this is impossible, at least approximate a local minimum. 

In the case of least squares linear regression we use $f_{\beta} (x) = \beta \cdot x$ as our class of functions and $\ell(\beta) = \sum_1^n |y_i - x_i \cdot \beta|^2$ as our loss function.

**Regularization** is modifying our loss function to penalize solutions which are "extreme" in some way.

We already saw two common regularization methods in lecture this week: "Ridge Regression" which adds a term of the form $\lambda |\beta|^2$ to the loss and "Lasso Regression" which adds a term of the form $\lambda |\beta|$ to the loss.  Both methods "penalize" solutions with large parameters.

In this notebook we will 

*  See how both methods can be viewed as maximum a posteriori probability estimates of the parameters given different priors.
*  See how using linear regression with a ridge penalty is equivalent to having $p$ "pseudo-observations" of a particularly nice form.
*  See how using linear regression with a ridge penalty is related to PCA linear regression.
    * PCA linear regression "throws away" singular values less than a given cut-off value.
    * Ridge regression reduces all singular values, but shrinks smaller singular values more.  So it can be seen as a "smooth" version of PCA.


## Ridge Regularization (for linear regression) as MAP with Gaussian Priors

The assumption of linear regression is that

$$Y = X \cdot \beta + \epsilon$$

where $\epsilon \sim \mathcal{N}(0, \sigma^2)$.

One way to formulate our desire for the parameters $\beta$ to be small is to suppose that $\beta \sim \mathcal{N}(0, \frac{1}{\lambda} I_p)$ (the multivariate Gaussian with with mean $0$ and variance matrix $\frac{1}{\lambda} I_p$).  Here $\lambda$ is just some positive constant.  When we choose a larger $\lambda$ we are expressing greater confidence in our guess that $\beta$ is small.  

Let's use the following notation for the pdf of $\mathcal{N}(0, t^2)$:

$$\phi_{t^2}(x) = \frac{1}{\sqrt{2\pi t^2}}\exp\left( - \frac{x^2}{2 t^2} \right)$$

Then we have


$$
\begin{align*}
P(\beta, \sigma^2 | \textrm{Data}) 
&\propto P(\textrm{Data} | \beta) P(\beta) \\
&= \left(\prod_1^n \phi_{\sigma^2}(y_i - x_i \cdot \beta) \right) \phi_{\frac{1}{\lambda}}(|\beta|)\\
\end{align*}
$$

As usual we take the negative log likelihood to get $\ell$.

$$
\begin{align*}
\ell (\beta) 
&= \frac{n+1}{2}\log(2\pi \sigma^2) + \frac{1}{2\sigma^2} \left[ \sum_{i=1}^n \left(y_i - x \cdot \beta\right)^2 + \frac{\lambda}{2} |\beta|^2 \right]
\end{align*}
$$

So (for any fixed $\sigma^2$) we have 

$$
\hat{\beta} = \mathop{\arg \min}\limits_{\beta}  \left( \sum_{i=1}^n \left(y_i - x_i \cdot \beta\right)^2 \right)+ \lambda |\beta|^2
$$

This shows that ridge regression (for linear regression) is just MAP with a normal prior for our parameters!

## Lasso Regularization (for linear regression) as MAP with Laplacian priors

Let's make $\beta_i \sim \operatorname{Laplace}(0, \frac{2}{\lambda})$ instead, where $\operatorname{Laplace}$ is the [Laplace distribution](https://en.wikipedia.org/wiki/Laplace_distribution).  It's pdf is 

$$f(t) = \lambda \exp(-\frac{1}{2} \lambda |t|)$$


The only thing that changes in our analysis above is that our "extra term" is 

$$
-  \sum_{j = 1}^{p} \log\left( \lambda \exp(-\frac{1}{2} \lambda \beta_i) \right) = -p\log(\lambda) + \frac{\lambda}{2}  \sum_{j = 1}^{p} |\beta_i|
$$

Remembering that our $\lambda$ is fixed (so the term $-p\log(\lambda)$ doesn't impact our minimization problem ) we have

$$
\hat{\beta} = \mathop{\arg \min}\limits_{\beta}  \left( \sum_{i=1}^n \left(y_i - x_i \cdot \beta\right)^2 \right) + \lambda \sum_{j = 1}^{p} |\beta_i|
$$

This shows that lasso regression (for linear regression) is just MAP with a Laplace prior for our parameters!

## Ridge Regression as pseudo-observations

Elements of Statistical Learning exercise 3.12 gives another way to understand ridge regression.

![Problem 3.12 of ESL](math_hour_assets/ESL-3.12.png)

Let's first see why this works, and then discuss a bit about the intuition here.


First we need to see how to find $\hat{\beta}_{\textrm{ridge}}$ explicitly.

$$
\ell_\lambda (\beta) = |y - X \beta|^2 + \lambda |\beta|^2
$$

Note that $\ell_\lambda$ is convex because it is the sum of two convex functions.  So a stationary point will be the global minimum.

Taking gradients:

$$
\begin{align*}
\nabla \ell_\lambda \big \vert_{\beta} 
&= -2X^\top (y - X \beta) + 2\lambda \beta\\
&= 2(X^\top X + \lambda I) \beta - 2X^\top y
\end{align*}
$$

so $\nabla \ell_\lambda \big \vert_{\beta_{\textrm{ridge}}} = 0$ has solution

$$
\beta_{\textrm{ridge}} = (X^\top X + \lambda I)^{-1} X^\top y
$$

It is worth noting that while $X^\top X$ is only invertible if $X$ has linearly independent columns, $X^\top X + \lambda I$ is a positive definite symmetric matrix, and is thus invertible without any assumptions on $X$.

Let $X_{\textrm{aug}}$ be the $(n+p) \times p$ matrix  $\begin{bmatrix} X \\ \sqrt{\lambda} I_p \end{bmatrix}$, where I am using vertical juxtaposition to indicate how these are stacked.  Let $y_{\textrm{aug}}$ be the $(n+p)$ vector $\begin{bmatrix} y \\ 0_p\end{bmatrix}$.

Then the ordinary least squares regression of  $y_{\textrm{aug}}$ on $X_{\textrm{aug}}$ gives us:

$$
\begin{align*}
\beta_{\textrm{aug}} 
&=  (X_{\textrm{aug}}^\top X_{\textrm{aug}})^{-1}X_{\textrm{aug}}^\top y_{\textrm{aug}}\\
&=  (\begin{bmatrix} X^\top &  \sqrt{\lambda} I_p \end{bmatrix} \begin{bmatrix} X \\ \sqrt{\lambda} I_p \end{bmatrix})^{-1}\begin{bmatrix} X^\top &  \sqrt{\lambda} I_p \end{bmatrix} \begin{bmatrix} y \\ 0_p\end{bmatrix}\\
&= (X^\top X + \lambda I_p)^{-1} X^\top y\\
&= \beta_{\textrm{ridge}}
\end{align*}
$$

This gives a completely different reason for being interested in the ridge regression solution:

If your design matrix $X$ had linearly dependent columns (or just "nearly" linearly dependent), then $X^\top X$ would be non-invertible (or, if there is a "near" linear dependence it will have a very bad "condition number").  One way to address this issue would be to invent some fake data to make the columns linearly independent.  A very natural choice is to invent one observation (row) for each feature where only that feature is non-negative, and where the response variable is set to 0 for this observation.  This makes sure the columns are linearly independent!  These observations also clearly bias the estimate of the coefficients towards zero.

Choosing to use the same constant $\sqrt{\lambda}$ for all of these "psuedo-observations" gives us ridge regression.

It is interesting to think about how this approach could be tailored to particular situations:

* If you know that the columns of $X$ are linearly independent aside from a subset of columns, you might be able to get away with just inventing "fake data" for these columns.
* If you care about some features more than others you might use a non-constant diagonal matrix with entries $\sqrt{\lambda_i}$ in place of the diagonal one.   This corresponds to minimizing the loss function $|y - X\beta|  + \sum_1^p \lambda_i |\beta_i|^2$
* Both of these correspond to minimizing a loss function of the form $|y - X \beta|^2 + |W \beta|^2$ where $W$ is a diagonal matrix.  One could also generalize by having $W$ be an arbitrary matrix.  This would correspond to augmenting the design matrix by $W^\top W$!

## Using SVD to understand the impact of $L_2$ regularization.

We can use the SVD to gain a little insight into what ridge regression is doing.

Let $X = U \Sigma V^\top$ with $V$ an orthogonal $p \times p$ matrix, $\Sigma$ a diagonal $p \times p$ matrix, and $U$ an orthogonal $n \times p$ matrix.  Note that this is a bit different from the factorizations we have used so far:  we are using all of the right singular vectors, but we are leaving out any left singular vectors beyond the $p^{th}$ one.  This is fine:  if you remember from the construction, those are all perpendicular to the image of $X$ and were chosen only to complete a basis of $\mathbb{R}^n$.

Then we have

$$
\begin{align*}
\beta_{\textrm{ridge}}
&= (X^\top X + \lambda I_p)^{-1} X^\top y\\
&= ( V \Sigma U^\top U \Sigma V^\top + \lambda I_p)^{-1} V\Sigma U^\top y\\
&= ( V \Sigma^2 V^\top + \lambda V V^\top)^{-1} V \Sigma U^\top y\\
&= ( V (\Sigma^2 + \lambda I_p) V^\top)^{-1} V \Sigma U^\top y\\
&=  V (\Sigma^2 + \lambda I_p)^{-1} V^\top V \Sigma U^\top y\\
&=  V (\Sigma^2 + \lambda I_p)^{-1} \Sigma U^\top y\\
\end{align*}
$$

Setting $\lambda = 0$ we obtain the OLS solution

$$
\beta_{\textrm{ols}} = V \Sigma^{-1} U^\top y
$$

So the coefficients for the $\beta_{\textrm{ols}}$ with respect to the basis $V$ are $\Sigma^{-1} U^\top y$, while the coefficients for $\beta_{\textrm{ridge}}$ with respect to the same basis are $(\Sigma^2 + \lambda I_p)^{-1} \Sigma U^\top y$.  These are the same, except that the coefficient of $v_j$ has been scaled by $\frac{\sigma_j^2}{\sigma_j^2 + \lambda}$. Notice that $v_j$ which have *larger* singular values are "shrunk" less than $v_j$ which have smaller singular values.  

This should make some intuitive sense.  If the right singular vectors with larger singular values are carrying "more of the signal" about the data $X$, then when we are "on an $L_2$ budget" we will want to prioritize those.

We can also apply $X$ to the ridge solution to see how to compare $\hat{y}_{\textrm{ridge}}$ vs. $\hat{y}_{\textrm{ols}}$:

$$
\begin{align*}
\hat{y}_{\textrm{ridge}} 
&= X\beta_{\textrm{ridge}}\\
&=  U \Sigma V^\top V (\Sigma^2 + \lambda I_p)^{-1} \Sigma U^\top y\\
&=  U \Sigma^2 (\Sigma^2 + \lambda I_p)^{-1} U^\top y\\
\end{align*}
$$

vs.

$$
\hat{y}_{\textrm{ols}} = U U^\top y
$$

So again, you scale the $u_j$ coordinates of the OLS solution by $\frac{\sigma_j^2}{\sigma_j^2 + \lambda}$ to obtain the ridge regression coordinates.

This gives yet another interpretation of why Ridge Regression is helpful:  it can be thought of as a kind of "smooth version" of PCA instead of just discarding small singular values we instead shrink smaller principle components more than large principle components.